In [1]:
import pandas as pd
import numpy as np

In [2]:
drugAnnot = pd.read_csv('../DrugCell/data/smiles_by_nsc.csv', index_col=0).dropna()
drugAnnot.head()

,SMILES
1,CC1=CC(=O)C=CC1=O
17,CCCCCCCCCCCCCCCc1cc(O)ccc1N
89,CN(C)CCC(=O)c1ccccc1
185,C[C@H]1C[C@H](C)C(=O)[C@@H](C1)[C@H](O)CC2CC(=...
295,OC(=O)CCCc1ccccc1


In [3]:
nci60Act = pd.read_csv('../DrugCell/data/nci60Act.csv', index_col=0)
nci60Act = nci60Act.rename(
    columns={
        'BR:HS 578T': 'BR:HS 578 T', 
        'BR:T-47D': 'BR:T47D',
        'CNS:SF-295': 'CNS:SF295',
        'CNS:SNB-19': 'CNS:SNB19',
        'CO:HT29': 'CO:HT 29',
        'CO:SW-620': 'SW620',
        'LE:HL-60(TB)': 'LE:HL-60',
        'ME:LOX IMVI': 'ME:LOXIMVI',
        'LC:A549/ATCC': 'LC:A549',
        'OV:IGROV1': 'OV:IGROV 1'
    }
)
nci60Act = nci60Act.drop([
    'CNS:SF-268', 'CNS:SF-539', 'CNS:SNB-75', 
    'CO:HCC-2998', 'LE:CCRF-CEM', 'ME:M14',
    'ME:MDA-N', 'LC:EKVX', 'LC:HOP-62', 
    'LC:HOP-92', 'OV:OVCAR-5', 'OV:NCI/ADR-RES',
    'RE:RXF 393', 'RE:SN12C', 'RE:TK-10', 'RE:UO-31'
], axis=1)

In [4]:
col = []
for i in nci60Act.columns:
    tmp = i.split(':')
    if len(tmp) == 2:
        col.append(tmp[1])
    else:
        col.append(tmp[0])

In [5]:
nci60Act.columns = col

In [6]:
sanger_cell_lines = pd.read_table('../DrugCell/data/sanger_cell_lines.txt')[[
    'Sanger name', 'CCLE Label'
]].dropna().reset_index(drop=True)

In [7]:
ccle_dict = {
    i.upper():j for i,j in zip(
        sanger_cell_lines['Sanger name'], 
        sanger_cell_lines['CCLE Label']
    )
}

In [8]:
nci60Act.columns = [
    ccle_dict[i.replace(' ', '-')] for i in nci60Act.columns
]

In [9]:
base = pd.DataFrame()
for i in nci60Act.columns:
    tmp = pd.DataFrame((nci60Act[i].dropna()))
    tmp.columns = ['Cell Line']
    tmp['Cell Line'] = [i]*len(tmp)
    tmp = pd.merge(tmp, drugAnnot, left_index=True, right_index=True)
    base = pd.concat([base, tmp])
    
base = base[['Cell Line', 'SMILES']]
base = base.reset_index(drop=True)

In [10]:
base.to_csv(
    '../DrugCell/data/rcellminer_test.txt', sep='\t', 
    header=None, index=None
)